In [1]:
import datasets
from datasets import load_dataset
squad  = load_dataset('squad_v2', split='train')
squad.shape

c:\Users\bhara\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset squad_v2 (C:/Users/bhara/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


(130319, 5)

In [2]:
squad[-1]

{'id': '5a7e070b70df9f001a87543d',
 'title': 'Matter',
 'context': 'The term "matter" is used throughout physics in a bewildering variety of contexts: for example, one refers to "condensed matter physics", "elementary matter", "partonic" matter, "dark" matter, "anti"-matter, "strange" matter, and "nuclear" matter. In discussions of matter and antimatter, normal matter has been referred to by Alfvén as koinomatter (Gk. common matter). It is fair to say that in physics, there is no broad consensus as to a general definition of matter, and the term "matter" usually is used in conjunction with a specifying modifier.',
 'question': 'What field of study has a variety of unusual contexts?',
 'answers': {'text': [], 'answer_start': []}}

In [3]:
from sentence_transformers import InputExample
#input example is just a format used for while training with sentence transformers
from tqdm import tqdm

train = []
for row in tqdm(squad):
    train.append(InputExample(texts = [row['question'], row['context']]))

100%|██████████| 130319/130319 [00:06<00:00, 19530.54it/s]


In [4]:
from sentence_transformers import datasets

batch_size = 24

loader = datasets.NoDuplicatesDataLoader(
    train, batch_size=batch_size
)

In [5]:
from sentence_transformers import models, SentenceTransformer
bert = models.Transformer('microsoft/mpnet-base')
pooler = models.Pooling(bert.get_word_embedding_dimension(), pooling_mode_mean_tokens=True) #why pooling?
model = SentenceTransformer(modules=[bert, pooler])
model

Some weights of the model checkpoint at microsoft/mpnet-base were not used when initializing MPNetModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing MPNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [6]:
from sentence_transformers import losses
loss = losses.MultipleNegativesRankingLoss(model)


In [7]:
epochs = 1 # more than 1 may lead to overfitting when using transformers
warmup = len(loader) * epochs /10
warmup

542.9

In [9]:
model.fit(train_objectives=[(loader, loss)], epochs=epochs, warmup_steps=warmup,
          output_path='mpnet-mnr-squad2', show_progress_bar=True)


Epoch:   0%|          | 0/1 [45:15<?, ?it/s]


KeyboardInterrupt: 